In [1]:
import logging
logging.getLogger("scapy.runtime").setLevel(logging.ERROR)
from scapy.all import *

In [3]:
sample_irc = "../data/SkypeIRC.cap"
pkts = sniff(offline=sample_irc)

In [4]:
pkts

<Sniffed: TCP:1150 UDP:1072 ICMP:23 Other:18>

In [5]:
pkts.show()

0000 Ether / IP / TCP 192.168.1.2:amt_blc_port > 212.204.214.114:6667 PA / Raw
0001 Ether / IP / TCP 212.204.214.114:6667 > 192.168.1.2:amt_blc_port A
0002 Ether / IP / TCP 212.204.214.114:6667 > 192.168.1.2:amt_blc_port PA / Raw
0003 Ether / IP / TCP 192.168.1.2:amt_blc_port > 212.204.214.114:6667 A
0004 Ether / IP / UDP / DNS Qry "2.1.168.192.in-addr.arpa." 
0005 Ether / IP / UDP / DNS Qry "114.214.204.212.in-addr.arpa." 
0006 Ether / IP / UDP / DNS Ans 
0007 Ether / IP / UDP / DNS Ans "sterling.freenode.net." 
0008 Ether / IP / UDP / DNS Qry "sterling.freenode.net." 
0009 Ether / IP / UDP / DNS Ans "212.204.214.114" 
0010 Ether / IP / UDP / DNS Qry "1.1.168.192.in-addr.arpa." 
0011 Ether / IP / UDP / DNS Ans "voyager.home." 
0012 Ether / IP / UDP / DNS Qry "voyager.home." 
0013 Ether / IP / UDP / DNS Ans "192.168.1.1" 
0014 Ether / IP / TCP 71.10.179.129:14232 > 192.168.1.2:as_debug PA / Raw
0015 Ether / IP / TCP 192.168.1.2:as_debug > 71.10.179.129:14232 A
0016 Ether / IP / TCP 192

In [6]:
pkts.summary(prn=lambda x:x.sprintf("{IP:%IP.src% -> %IP.dst%\n}{Raw:%Raw.load%\n}"))

192.168.1.2 -> 212.204.214.114
'ISON Thunfisch Smiley SmileyG\n'

212.204.214.114 -> 192.168.1.2

212.204.214.114 -> 192.168.1.2
':sterling.freenode.net 303 vmlemon :SmileyG \r\n'

192.168.1.2 -> 212.204.214.114

192.168.1.2 -> 192.168.1.1

192.168.1.2 -> 192.168.1.1

192.168.1.1 -> 192.168.1.2

192.168.1.1 -> 192.168.1.2

192.168.1.2 -> 192.168.1.1

192.168.1.1 -> 192.168.1.2

192.168.1.2 -> 192.168.1.1

192.168.1.1 -> 192.168.1.2

192.168.1.2 -> 192.168.1.1

192.168.1.1 -> 192.168.1.2

71.10.179.129 -> 192.168.1.2
'\x88\xd3\xcdC\xf5\xa0\x0c\xc7\x0e\xc7\x94Wj\x97\xcb\xa7a\x84!@\x00\x00\x00\x00\xff\xff\x02'

192.168.1.2 -> 71.10.179.129

192.168.1.2 -> 71.10.179.129
'\xe2t\xf3P=(\xc3\xb1\xc31\xe5\x95\xda\xe5\xf2i\xd8\x15\x01\x00\x00\x00\xff\xff'

212.204.214.114 -> 192.168.1.2
':yaloki!n=yaloki@156.185-64-87.adsl-dyn.isp.belgacom.be PRIVMSG #amarok :+how do you mean\r\n'

192.168.1.2 -> 212.204.214.114

71.10.179.129 -> 192.168.1.2

192.168.1.2 -> 192.168.1.1

192.168.1.1 -> 192.168.1.

In [7]:
def filter_pkt_by_query(pkt, query):
    try:
        raw_load = pkt.getlayer(Raw).fields.get("load")
        if query in raw_load:
            msg = ("*****************\nQUERY FOUND\nFrom {IP:%IP.src% -> %IP.dst%\n}*****************\n")
            print pkt.sprintf(msg)
            print raw_load
    except Exception:
        pass

In [8]:
for pkt in pkts:
    filter_pkt_by_query(pkt, "amarok")

*****************
QUERY FOUND
From 212.204.214.114 -> 192.168.1.2
*****************

:yaloki!n=yaloki@156.185-64-87.adsl-dyn.isp.belgacom.be PRIVMSG #amarok :+how do you mean

*****************
QUERY FOUND
From 212.204.214.114 -> 192.168.1.2
*****************

:jefferai!n=jefferai@amarok/developer/mitchell PRIVMSG #amarok :+eh?

*****************
QUERY FOUND
From 212.204.214.114 -> 192.168.1.2
*****************

:jefferai!n=jefferai@amarok/developer/mitchell PRIVMSG #amarok :+what, what

*****************
QUERY FOUND
From 212.204.214.114 -> 192.168.1.2
*****************

:hurra!n=lolo@p54921341.dip0.t-ipconnect.de PRIVMSG #amarok :+;p

*****************
QUERY FOUND
From 212.204.214.114 -> 192.168.1.2
*****************

:sterling.freenode.net 352 vmlemon #rokymotion n=lando pool-71-121-181-242.sttlwa.dsl-w.verizon.net irc.freenode.net land0 H :0 gaim
:sterling.freenode.net 352 vmlemon #rokymotion n=insane amarok/bot/insanity irc.freenode.net insanity H :0 Ruby bot. (c) Tom Gilbert
:ster

It's a bit more difficult to sniff Jabber/XMPP traffic (more often over TLS).  We can tell it's traffic from Jabber but the actual conversations (the Raw portion of the layer) is encrypted.  

But - if for whatever reason you have the SSL keys to this conversation (like, _you_ are one of the persons in the conversation, or some ominous government institution with super-smart cryptographers), you can decrypt the conversation.  It's not possible in Scapy right now, but there's another Python package that plugs into Scapy, [scapy-ssl_tls](https://github.com/tintinweb/scapy-ssl_tls) (disclaimer: have not used it yet).  It's far easier in Wireshark, with a nice tutorial written [here](http://support.citrix.com/article/CTX116557).

But let's just take a look at some Jabber packets real quick:

In [14]:
jabber = "../data/xmpp.cap"
jpkts = sniff(offline=jabber)

In [15]:
jpkts

<Sniffed: TCP:39 UDP:0 ICMP:0 Other:0>

In [16]:
jpkts[3].show()

###[ Ethernet ]###
  dst       = 00:0e:b6:29:18:89
  src       = 00:90:f5:aa:83:da
  type      = 0x800
###[ IP ]###
     version   = 4L
     ihl       = 5L
     tos       = 0x0
     len       = 181
     id        = 62087
     flags     = DF
     frag      = 0L
     ttl       = 64
     proto     = tcp
     chksum    = 0xb14c
     src       = 192.168.97.167
     dst       = 208.68.163.218
     \options   \
###[ TCP ]###
        sport     = 46553
        dport     = jabber_client
        seq       = 1190763336
        ack       = 2831350836
        dataofs   = 8L
        reserved  = 0L
        flags     = PA
        window    = 229
        chksum    = 0x9716
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (688351713, 3452727006))]
###[ Raw ]###
           load      = "<stream:stream to='jabber.org' xmlns:stream='http://etherx.jabber.org/streams' xmlns='jabber:client' xml:lang='en' version='1.0'>"


In [17]:
jpkts[5].show()

###[ Ethernet ]###
  dst       = 00:90:f5:aa:83:da
  src       = 00:0e:b6:29:18:89
  type      = 0x800
###[ IP ]###
     version   = 4L
     ihl       = 5L
     tos       = 0x60
     len       = 418
     id        = 37572
     flags     = DF
     frag      = 0L
     ttl       = 53
     proto     = tcp
     chksum    = 0x1ac3
     src       = 208.68.163.218
     dst       = 192.168.97.167
     \options   \
###[ TCP ]###
        sport     = jabber_client
        dport     = 46553
        seq       = 2831350836
        ack       = 1190763465
        dataofs   = 8L
        reserved  = 0L
        flags     = PA
        window    = 122
        chksum    = 0x1e70
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (3452727031, 688351713))]
###[ Raw ]###
           load      = "<?xml version='1.0'?><stream:stream xmlns='jabber:client' xmlns:stream='http://etherx.jabber.org/streams' from='jabber.org' id='edd1f8b132086edc' version='1.0'><stream:features><startt